In [ ]:
train_dir="/content/Data/Train"
test_dir="/content/Data/Test"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from helper_functions import plot_loss_curves,unzip_data,walk_through_dir,compare_historys

In [ ]:
unzip_data("/content/drive/MyDrive/Data-20240304T162637Z-001.zip")


In [ ]:
walk_through_dir("/content/Data")
class_names=

There are 2 directories and 0 images in '/content/Data'.
There are 2 directories and 0 images in '/content/Data/Test'.
There are 0 directories and 42 images in '/content/Data/Test/Real'.
There are 0 directories and 42 images in '/content/Data/Test/AI'.
There are 2 directories and 0 images in '/content/Data/Train'.
There are 0 directories and 394 images in '/content/Data/Train/Real'.
There are 0 directories and 497 images in '/content/Data/Train/AI'.


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
train_ds=image_dataset_from_directory(train_dir,
                                      label_mode='binary',
                                      batch_size=32,
                                      shuffle=True,
                                      )

test_ds=image_dataset_from_directory(test_dir,
                                     label_mode='binary',
                                     batch_size=32,
                                     shuffle=True)

Found 888 files belonging to 2 classes.
Found 84 files belonging to 2 classes.


In [ ]:
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.Resizing(256,256),
    tf.keras.layers.Rescaling(1/255.),
    tf.keras.layers.RandomTranslation(0.2,0.2)
])

In [ ]:
base_model=tf.keras.applications.vgg19.VGG19( include_top=False)
base_model.trainable=False
Input=tf.keras.layers.Input(shape=(256,256,3))
x=data_augmentation(Input)
x=base_model(Input,training=False)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
Output=tf.keras.layers.Dense(1,activation='sigmoid')(x)
model=tf.keras.Model(Input,Output)
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
history_1=model.fit(train_ds,
                    epochs=10,
                    steps_per_epoch=len(train_ds),
                    validation_data=test_ds,
                    validation_steps=len(test_ds))

Epoch 1/10
28/28 [==============================] - 41s 930ms/step - loss: 2.0530 - accuracy: 0.5833 - val_loss: 1.4229 - val_accuracy: 0.5833
Epoch 2/10
28/28 [==============================] - 21s 529ms/step - loss: 1.3825 - accuracy: 0.6486 - val_loss: 1.1475 - val_accuracy: 0.6190
Epoch 3/10
28/28 [==============================] - 19s 532ms/step - loss: 1.0886 - accuracy: 0.6824 - val_loss: 0.9415 - val_accuracy: 0.6429
Epoch 4/10
28/28 [==============================] - 20s 527ms/step - loss: 0.8748 - accuracy: 0.7331 - val_loss: 0.7628 - val_accuracy: 0.6548
Epoch 5/10
28/28 [==============================] - 20s 531ms/step - loss: 0.7662 - accuracy: 0.7511 - val_loss: 0.8360 - val_accuracy: 0.6667
Epoch 6/10
28/28 [==============================] - 21s 553ms/step - loss: 0.6668 - accuracy: 0.7736 - val_loss: 0.6827 - val_accuracy: 0.7500
Epoch 7/10
28/28 [==============================] - 20s 517ms/step - loss: 0.5960 - accuracy: 0.7894 - val_loss: 0.7408 - val_accuracy: 0.7262

In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 20024897 (76.39 MB)
Trainable params: 20024897 (76.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.layers[1].name

'vgg19'

In [ ]:
base_model=model.layers[1]
base_model.trainable=True
for i,layer in enumerate(base_model.layers[:-7]):
  layer.trainable=False


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam( learning_rate=0.0001),
              metrics=['accuracy'])
model.fit(train_ds,
          epochs=history_1.epoch[-1]+10,
          steps_per_epoch=len(train_ds),
          validation_data=test_ds,
          validation_steps=len(test_ds),
          initial_epoch=history_1.epoch[-1]
          )

Epoch 10/19
28/28 [==============================] - 27s 634ms/step - loss: 1.0768 - accuracy: 0.6486 - val_loss: 0.5911 - val_accuracy: 0.7500
Epoch 11/19
28/28 [==============================] - 20s 555ms/step - loss: 0.5725 - accuracy: 0.6959 - val_loss: 0.5547 - val_accuracy: 0.7024
Epoch 12/19
28/28 [==============================] - 21s 552ms/step - loss: 0.4781 - accuracy: 0.7736 - val_loss: 0.5280 - val_accuracy: 0.7381
Epoch 13/19
28/28 [==============================] - 21s 554ms/step - loss: 0.3932 - accuracy: 0.8311 - val_loss: 0.5743 - val_accuracy: 0.7381
Epoch 14/19
28/28 [==============================] - 20s 552ms/step - loss: 0.3025 - accuracy: 0.8727 - val_loss: 0.7648 - val_accuracy: 0.6905
Epoch 15/19
28/28 [==============================] - 21s 555ms/step - loss: 0.3318 - accuracy: 0.8637 - val_loss: 0.5658 - val_accuracy: 0.7619
Epoch 16/19
28/28 [==============================] - 21s 579ms/step - loss: 0.2212 - accuracy: 0.9054 - val_loss: 0.6807 - val_accuracy:

In [ ]:
model.save('/content/drive/MyDrive/Models/VGG.h5')

In [ ]:
model_2=tf.keras.models.load_model('/content/drive/MyDrive/Models/VGG')

In [ ]:
model_2.evaluate(test_ds)

3/3 [==============================] - 75s 23s/step - loss: 0.6322 - accuracy: 0.8452


[0.6321544647216797, 0.8452380895614624]

In [ ]:
model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 20024897 (76.39 MB)
Trainable params: 11799553 (45.01 MB)
Non-trainable params: 8225344 (31.38 MB)
_________________________________________________________________


In [ ]:
import pathlib
import numpy as np
dir=pathlib.Path(train_dir)
class_names=np.array(sorted([i.name for i in dir.glob('*')]))
class_names

array(['AI', 'Real'], dtype='<U4')

In [ ]:
img=tf.io.read_file("/.jpg")
img=tf.io.decode_image(img)
img=tf.image.resize(img,[256,256])
class_names[int(tf.squeeze(tf.round(model_2.predict(tf.expand_dims(img,axis=0)))).numpy())]

1/1 [==============================] - 1s 810ms/step


'Real'